# 📘 Engineering-Grade Pandas Cheatsheet

## Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

print(pd.__version__)
print(np.__version__)

## DataFrame Creation & Schema Handling

In [ ]:
# From dictionary
df_dict = pd.DataFrame({"id": [1, 2, 3], "value": [10.5, 20.1, 30.6]})

# From numpy array
arr = np.random.randn(5, 3)
df_np = pd.DataFrame(arr, columns=["A", "B", "C"])

# From Arrow Table
table = pa.table({"x": [1,2,3], "y": ["a","b","c"]})
df_arrow = table.to_pandas()

# Schema enforcement
df_schema = df_dict.astype({"id": "int64", "value": "float32"})

## Indexing & Selection (Executive Grade)

In [ ]:
df = pd.DataFrame({
    "id": [101, 102, 103, 104],
    "dept": ["HR", "IT", "HR", "FIN"],
    "salary": [50000, 60000, 52000, 58000]
})

# loc, iloc
print(df.loc[df["dept"]=="HR", ["id","salary"]])
print(df.iloc[0:2, 1:])

# MultiIndex
df_m = df.set_index(["dept","id"])
print(df_m.xs("HR", level="dept"))

# Query with variable
threshold = 55000
print(df.query("salary > @threshold"))

## Data Cleaning & Transformation

In [ ]:
df = pd.DataFrame({
    "name": [" Alice ", "Bob", None, "Charlie"],
    "joined": ["2021-01-01", "2021/02/15", None, "2021-03-20"],
    "score": [np.nan, 8.5, 7.0, 9.1]
})

# Strip whitespace
df["name"] = df["name"].str.strip()

# Handle missing values
df["name"].fillna("Unknown", inplace=True)
df["score"] = df["score"].interpolate()

# Convert dates
df["joined"] = pd.to_datetime(df["joined"], errors="coerce")

## Merging, Joins & Reshaping

In [ ]:
left = pd.DataFrame({"id":[1,2,3], "dept":["HR","IT","FIN"]})
right = pd.DataFrame({"id":[2,3,4], "salary":[60,70,80]})

# Inner join
print(pd.merge(left, right, on="id", how="inner"))

# Outer join
print(pd.merge(left, right, on="id", how="outer"))

# Anti-join
anti = left[~left["id"].isin(right["id"])]
print(anti)

# Reshaping
df = pd.DataFrame({
    "id":[1,2,3],
    "Q1":[100,200,300],
    "Q2":[110,210,310]
})
melted = df.melt(id_vars="id", var_name="quarter", value_name="amount")
print(melted)

## GroupBy & Aggregations

In [ ]:
df = pd.DataFrame({
    "dept":["HR","HR","IT","IT","FIN"],
    "salary":[50,60,70,65,80],
    "bonus":[5,6,7,6,8]
})

# Multiple aggs with rename
agg = df.groupby("dept").agg(
    avg_salary=("salary","mean"),
    max_bonus=("bonus","max")
)
print(agg)

# Transform vs apply
df["pct_salary"] = df.groupby("dept")["salary"].transform(lambda x: x/x.sum())

## Time Series & Window Functions

In [ ]:
rng = pd.date_range("2023-01-01", periods=6, freq="D")
ts = pd.DataFrame({"date":rng, "value":[10,12,15,14,13,18]}).set_index("date")

# Resample
print(ts.resample("2D").sum())

# Rolling average
print(ts.rolling(window=3).mean())

# Expanding sum
print(ts.expanding().sum())

## Performance Optimization

In [ ]:
df = pd.DataFrame({
    "a": np.random.randint(0,1000,1000000),
    "b": np.random.rand(1000000)
})

# Memory usage
print(df.info(memory_usage="deep"))

# Downcast
df["a"] = pd.to_numeric(df["a"], downcast="unsigned")
df["b"] = pd.to_numeric(df["b"], downcast="float")

# Query optimization
res = df.query("a < 10 and b > 0.5")

## Validation & Testing

In [ ]:
df = pd.DataFrame({
    "id":[1,2,3,3],
    "value":[10,20,None,40]
})

# Duplicate check
assert df["id"].is_unique == False

# Null check
print(df["value"].isnull().sum())

# Type check
from pandas.api.types import is_numeric_dtype
print(is_numeric_dtype(df["value"]))

## Interoperability

In [ ]:
# SQL (example requires SQLAlchemy)
# from sqlalchemy import create_engine
# engine = create_engine("sqlite:///:memory:")
# df.to_sql("table1", engine)
# pd.read_sql("select * from table1", engine)

# Pandas <-> Arrow
df = pd.DataFrame({"a":[1,2,3]})
tbl = pa.Table.from_pandas(df)
df2 = tbl.to_pandas()

## Debugging & Engineering Practices

In [ ]:
df = pd.DataFrame({
    "col1": np.random.choice(["A","B","C"], size=100),
    "col2": np.random.randn(100)
})

# Memory usage
print(df.info(memory_usage="deep"))

# Descriptive stats
print(df.describe(include="all"))

# Profiling (if installed)
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df)
# profile.to_notebook_iframe()